# Examples
Some example code for working with the ooiopendata storage account.

## Load available keys

In [ ]:
import ooiod

In [ ]:
key_file = '../secrets/tjcrone.pickle'
keys = ooiod.secrets.load_keys(key_file)

## List the first ten blobs in the camhd container

In [ ]:
from azure.storage.blob import BlockBlobService

In [ ]:
if 'camhd' in keys:
    camhd_service = BlockBlobService('ooiopendata', sas_token = keys['camhd'])
else:
    raise Exception('Camhd container key not available')

In [ ]:
blob_list = camhd_service.list_blobs('camhd')
for i in range(10):
    print(blob_list.items[i].name)

## Download a NetCDF file from OOI results server

In [ ]:
netcdf_url = 'https://opendap.oceanobservatories.org/async_results/crone@ldeo.columbia.edu/20191022T181459885Z-RS03ASHS-MJ03B-10-CTDPFB304-streamed-ctdpf_optode_sample/deployment0005_RS03ASHS-MJ03B-10-CTDPFB304-streamed-ctdpf_optode_sample_20191022T120127.028053-20191022T180126.225018.nc'
netcdf_filename = netcdf_url.split('/')[-1]

In [ ]:
import wget

In [ ]:
wget.download(netcdf_url);

## Write the original NetCDF file into Azure blob

In [ ]:
from azure.storage.blob import BlockBlobService

In [ ]:
if 'botpt' in keys:
    botpt_service = BlockBlobService('ooiopendata', sas_token = keys['botpt'])
else:
    raise Exception('Botpt container key not available')

In [ ]:
botpt_service.create_blob_from_path('botpt', netcdf_filename, netcdf_filename, validate_content=True);

In [ ]:
for blob in botpt_service.list_blobs('botpt'):
    print(blob.name)

## Load the file into Xarray and save to ooiopendata

In [ ]:
import xarray as xr
import zarr
import logging
logging.getLogger("azure.storage").setLevel(logging.CRITICAL)

In [ ]:
ds = xr.open_dataset(netcdf_filename)

In [ ]:
abs_store = zarr.storage.ABSStore('botpt', netcdf_filename[:-2] + 'zarr', account_name='ooiopendata',
                                  blob_service_kwargs={'sas_token':keys['botpt']})

In [ ]:
ds.to_zarr(abs_store);

## Remove local file

In [ ]:
import os
os.remove(local_file)

## List remote blobs using public interface

In [ ]:
def list_blob_set(storage_account, container, nextmarker = None):
    import requests
    from xml.etree import ElementTree
    
    request_uri = 'https://%s.blob.core.windows.net/%s?restype=container&comp=list' % (storage_account, container)
    if nextmarker:
        request_uri = request_uri + '&marker=' + nextmarker
    catalog_xml = requests.get(request_uri).content
    catalog_tree = ElementTree.fromstring(catalog_xml)
    blob_set = []
    for blob in catalog_tree.find('Blobs').findall('Blob'):
        blob_set.append(blob.find('Url').text)
    nextmarker = catalog_tree.find('NextMarker').text
    return blob_set, nextmarker

In [ ]:
def list_blobs(storage_account, container):
    blobs = []
    blob_set, nextmarker = list_blob_set('ooiopendata', 'camhd')
    blobs = blobs + blob_set
    while nextmarker:
        blob_set, nextmarker = list_blob_set('ooiopendata', 'camhd', nextmarker)
        blobs = blobs + blob_set
    return blobs

In [ ]:
%%time
blobs = list_blobs('ooiopendata', 'camhd')

In [ ]:
blobs[0]